In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-05-20 19:48:31.276778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 19:48:31.276875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 19:48:31.378815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# load data frames
df_train = pd.read_csv('/kaggle/input/dataframes/train.csv')
df_val = pd.read_csv('/kaggle/input/dataframes/val.csv')
df_test = pd.read_csv('/kaggle/input/dataframes/val.csv')

# load joint embeddings
#with open('/kaggle/input/v-embeds/v_embeds_train.pkl', 'rb') as f:
   # v_embeds_train = pickle.load(f) 
    
#with open('/kaggle/input/v-embeds/v_embeds_val.pkl', 'rb') as f:
  #  v_embeds_val = pickle.load(f) 
    
with open('/kaggle/input/j-embeds/j_embeds_test.pkl', 'rb') as f:
    j_embeds_test = pickle.load(f) 

In [3]:
print(len(j_embeds_test))

109


In [6]:
X_train = torch.stack(j_embeds_test[:90], dim=0).cpu().numpy()
X_val = torch.stack(j_embeds_test[90:100], dim=0).cpu().numpy()
X_test = torch.stack(j_embeds_test[100:], dim=0).cpu().numpy()
print(X_train.shape, X_val.shape, X_test.shape)

(90, 612, 768) (10, 612, 768) (9, 612, 768)


In [14]:
# convert labels to numerical values
label_encoder = LabelEncoder()
labels = df_test['AUTHOR'].tolist()
y_train = labels[:90]
y_val = labels[90:100]
y_test = labels[100:]

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)
print(y_test)

[2 2 4 5 3 8 8 4 6]


In [15]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(612, 768)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')



Epoch 1/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.0625 - loss: 2.2881

I0000 00:00:1716235031.305524     494 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716235031.322181     494 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.0607 - loss: 33.4969 - val_accuracy: 0.1000 - val_loss: 34.0372
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2896 - loss: 29.4345 - val_accuracy: 0.0000e+00 - val_loss: 40.3626
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3139 - loss: 22.8954 - val_accuracy: 0.2000 - val_loss: 19.4853
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4922 - loss: 8.9841 - val_accuracy: 0.2000 - val_loss: 16.0413
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6576 - loss: 5.5278 - val_accuracy: 0.0000e+00 - val_loss: 23.3076
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.7612 - loss: 3.2455 - val_accuracy: 0.0000e+00 - val_loss: 26.7728
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8803 - loss: 0.9271 - val_accuracy: 0.0000e+00 - val_loss: 27.1850
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9087 - loss: 1.2769 - val_accuracy: 0.0000e+00 - 

Besser als mit visuellen Features und über Chance Performance. Allerdings nur mit 109 Datensätzen trainiert. Wie kann man sichergehen, dass das Model nicht nur die Textfeatures benutzt? 